In [1]:
!pwd
!ls -la

/content
total 16
drwxr-xr-x 1 root root 4096 Jun 13 13:36 .
drwxr-xr-x 1 root root 4096 Jun 17 15:58 ..
drwxr-xr-x 4 root root 4096 Jun 13 13:36 .config
drwxr-xr-x 1 root root 4096 Jun 13 13:36 sample_data


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!cd "/content/drive/MyDrive/educational-course-content-generator-with-qna-bot-using-bedrock-main" && pwd && ls -la

/content/drive/MyDrive/educational-course-content-generator-with-qna-bot-using-bedrock-main
total 47
-rw------- 1 root root  1186 May  8 09:29 app.py
drwx------ 2 root root  4096 May  8 09:29 architecture_diagrams
-rw------- 1 root root  3867 May  8 09:29 cdk.json
-rw------- 1 root root   309 May  8 09:29 CODE_OF_CONDUCT.md
-rw------- 1 root root  3160 May  8 09:29 CONTRIBUTING.md
drwx------ 2 root root  4096 May  8 09:29 educational_course_content_generator_with_qna_bot_using_bedrock
drwx------ 2 root root  4096 May  8 09:29 kb_dataset
drwx------ 2 root root  4096 May  8 09:29 lambda
-rw------- 1 root root   947 May  8 09:29 LICENSE
-rw------- 1 root root   630 May  8 09:29 project_config.json
-rw------- 1 root root 12136 May  8 09:29 README.md
-rw------- 1 root root    14 May  8 09:29 requirements-dev.txt
-rw------- 1 root root   206 May  8 09:29 requirements.txt
-rw------- 1 root root  1361 Jun 17 15:59 Setup-GitHub-Actions.ipynb
-rw------- 1 root root   437 May  8 09:29 source.bat


In [4]:
!cd "/content/drive/MyDrive/educational-course-content-generator-with-qna-bot-using-bedrock-main" && \
mkdir -p .github/workflows && \
ls -la .github/

total 4
drwx------ 2 root root 4096 Jun 17 16:00 workflows


In [5]:
import os
os.chdir('/content/drive/MyDrive/educational-course-content-generator-with-qna-bot-using-bedrock-main')

workflow_content = """name: Deploy Bedrock Course Generator

on:
  push:
    branches: [ main ]
  pull_request:
    branches: [ main ]

env:
  AWS_REGION: us-east-1

# Permissions OIDC requises
permissions:
  id-token: write   # Requis pour demander le JWT OIDC
  contents: read    # Requis pour actions/checkout

jobs:
  deploy:
    runs-on: ubuntu-latest

    steps:
      - name: Checkout repository
        uses: actions/checkout@v4

      - name: Configure AWS credentials using OIDC
        uses: aws-actions/configure-aws-credentials@e3dd6a429d7300a6a4c196c26e071d42e0343502
        with:
          role-to-assume: arn:aws:iam::796973472728:role/github.to.aws.oicd
          role-session-name: github-actions-bedrock-session
          aws-region: ${{ env.AWS_REGION }}

      - name: Setup Python
        uses: actions/setup-python@v4
        with:
          python-version: '3.12'

      - name: Install dependencies
        run: |
          python -m venv .venv
          source .venv/bin/activate
          pip install -r requirements.txt

      - name: Install AWS CDK
        run: |
          npm install -g aws-cdk

      - name: CDK Bootstrap (if needed)
        run: |
          source .venv/bin/activate
          cdk bootstrap

      - name: CDK Synth
        run: |
          source .venv/bin/activate
          cdk synth --all

      - name: CDK Deploy
        run: |
          source .venv/bin/activate
          cdk deploy --all --require-approval never

      - name: Verify deployment
        run: |
          echo "Deployment completed successfully!"
          aws bedrock list-foundation-models --region ${{ env.AWS_REGION }} --max-items 1
"""

with open('.github/workflows/deploy.yml', 'w') as f:
    f.write(workflow_content)

print("✅ Fichier deploy.yml créé avec succès!")

✅ Fichier deploy.yml créé avec succès!


In [6]:
!cd "/content/drive/MyDrive/educational-course-content-generator-with-qna-bot-using-bedrock-main" && \
ls -la .github/workflows/ && \
echo "--- Contenu du fichier deploy.yml ---" && \
head -10 .github/workflows/deploy.yml

total 2
-rw------- 1 root root 1647 Jun 17 16:00 deploy.yml
--- Contenu du fichier deploy.yml ---
name: Deploy Bedrock Course Generator

on:
  push:
    branches: [ main ]
  pull_request:
    branches: [ main ]

env:
  AWS_REGION: us-east-1


In [7]:
import json

# Lire la configuration actuelle
with open('project_config.json', 'r') as f:
    config = json.load(f)

# Modifier pour réutiliser les ressources Genk01
config.update({
    "stack_name_prefix": "Genk01-BedrockCourse-",
    "reuse_existing": {
        "user_pool_id": "us-east-1_dFyTl7Ab1",
        "existing_bucket": "genk01-webappnestedstackweba-websitebucket4326d7c2-sto8uwvirujy",
        "existing_queue": "Genk01-UseCaseManagementDLQ",
        "parent_stack": "Genk01"
    },
    "environment": "genk01-extension"
})

# Sauvegarder
with open('project_config.json', 'w') as f:
    json.dump(config, f, indent=2)

print("✅ Configuration modifiée pour réutiliser Genk01!")
print(json.dumps(config, indent=2))

✅ Configuration modifiée pour réutiliser Genk01!
{
  "CloudFrontWAFStackName": "CloudFrontWAFStack",
  "CourseStackName": "CourseStack",
  "QnAStackName": "QnAStack",
  "existing_vpc_id": "",
  "model_id": "anthropic.claude-3-5-sonnet-20240620-v1:0",
  "embeddings_model_id": "amazon.titan-embed-text-v2:0",
  "qna_model_id": "anthropic.claude-3-5-sonnet-20240620-v1:0",
  "embeddings_vector_size": 1024,
  "vector_index_name": "couse-content-default-index",
  "metadata_field": "course_content_metadata",
  "text_field": "course_content_text_chunk",
  "vector_field": "couse-content-default-vector",
  "vpc": {
    "cidr_range": "10.0.0.0/16",
    "cidr_mask": 24
  },
  "stack_name_prefix": "Genk01-BedrockCourse-",
  "reuse_existing": {
    "user_pool_id": "us-east-1_dFyTl7Ab1",
    "existing_bucket": "genk01-webappnestedstackweba-websitebucket4326d7c2-sto8uwvirujy",
    "existing_queue": "Genk01-UseCaseManagementDLQ",
    "parent_stack": "Genk01"
  },
  "environment": "genk01-extension"
}


In [8]:
import json

# Lire la configuration actuelle
with open('project_config.json', 'r') as f:
    config = json.load(f)

print("Configuration originale :")
print(json.dumps(config, indent=2))
print("\n" + "="*50 + "\n")

# Modifier pour réutiliser les ressources Genk01
config.update({
    "stack_name_prefix": "Genk01-BedrockCourse-",
    "reuse_existing": {
        "user_pool_id": "us-east-1_dFyTl7Ab1",
        "existing_bucket": "genk01-webappnestedstackweba-websitebucket4326d7c2-sto8uwvirujy",
        "existing_queue_url": "https://sqs.us-east-1.amazonaws.com/796973472728/Genk01-UseCaseManagementSetupUseCaseMa-UseCaseManagementDLQ738693BC-fLN2YDAH7sxM",
        "parent_stack": "Genk01",
        "region": "us-east-1"
    },
    "environment": "genk01-extension",
    "cost_optimization": {
        "enable_auto_scaling": True,
        "use_on_demand_billing": True,
        "enable_resource_cleanup": True
    }
})

# Sauvegarder
with open('project_config.json', 'w') as f:
    json.dump(config, f, indent=2)

print("✅ Configuration MODIFIÉE pour réutiliser Genk01!")
print("\nNouvelle configuration :")
print(json.dumps(config, indent=2))

Configuration originale :
{
  "CloudFrontWAFStackName": "CloudFrontWAFStack",
  "CourseStackName": "CourseStack",
  "QnAStackName": "QnAStack",
  "existing_vpc_id": "",
  "model_id": "anthropic.claude-3-5-sonnet-20240620-v1:0",
  "embeddings_model_id": "amazon.titan-embed-text-v2:0",
  "qna_model_id": "anthropic.claude-3-5-sonnet-20240620-v1:0",
  "embeddings_vector_size": 1024,
  "vector_index_name": "couse-content-default-index",
  "metadata_field": "course_content_metadata",
  "text_field": "course_content_text_chunk",
  "vector_field": "couse-content-default-vector",
  "vpc": {
    "cidr_range": "10.0.0.0/16",
    "cidr_mask": 24
  },
  "stack_name_prefix": "Genk01-BedrockCourse-",
  "reuse_existing": {
    "user_pool_id": "us-east-1_dFyTl7Ab1",
    "existing_bucket": "genk01-webappnestedstackweba-websitebucket4326d7c2-sto8uwvirujy",
    "existing_queue": "Genk01-UseCaseManagementDLQ",
    "parent_stack": "Genk01"
  },
  "environment": "genk01-extension"
}


✅ Configuration MODIFIÉ

In [9]:
import json

# Lire la configuration actuelle
with open('project_config.json', 'r') as f:
    config = json.load(f)

print("Configuration originale :")
print(json.dumps(config, indent=2))
print("\n" + "="*50 + "\n")

# Modifier pour réutiliser les ressources Genk01
config.update({
    "stack_name_prefix": "Genk01-BedrockCourse-",
    "reuse_existing": {
        "user_pool_id": "us-east-1_dFyTl7Ab1",
        "existing_bucket": "genk01-webappnestedstackweba-websitebucket4326d7c2-sto8uwvirujy",
        "existing_queue_url": "https://sqs.us-east-1.amazonaws.com/796973472728/Genk01-UseCaseManagementSetupUseCaseMa-UseCaseManagementDLQ738693BC-fLN2YDAH7sxM",
        "parent_stack": "Genk01",
        "region": "us-east-1"
    },
    "environment": "genk01-extension",
    "cost_optimization": {
        "enable_auto_scaling": True,
        "use_on_demand_billing": True,
        "enable_resource_cleanup": True
    }
})

# Sauvegarder
with open('project_config.json', 'w') as f:
    json.dump(config, f, indent=2)

print("✅ Configuration MODIFIÉE pour réutiliser Genk01!")
print("\nNouvelle configuration :")
print(json.dumps(config, indent=2))

Configuration originale :
{
  "CloudFrontWAFStackName": "CloudFrontWAFStack",
  "CourseStackName": "CourseStack",
  "QnAStackName": "QnAStack",
  "existing_vpc_id": "",
  "model_id": "anthropic.claude-3-5-sonnet-20240620-v1:0",
  "embeddings_model_id": "amazon.titan-embed-text-v2:0",
  "qna_model_id": "anthropic.claude-3-5-sonnet-20240620-v1:0",
  "embeddings_vector_size": 1024,
  "vector_index_name": "couse-content-default-index",
  "metadata_field": "course_content_metadata",
  "text_field": "course_content_text_chunk",
  "vector_field": "couse-content-default-vector",
  "vpc": {
    "cidr_range": "10.0.0.0/16",
    "cidr_mask": 24
  },
  "stack_name_prefix": "Genk01-BedrockCourse-",
  "reuse_existing": {
    "user_pool_id": "us-east-1_dFyTl7Ab1",
    "existing_bucket": "genk01-webappnestedstackweba-websitebucket4326d7c2-sto8uwvirujy",
    "existing_queue_url": "https://sqs.us-east-1.amazonaws.com/796973472728/Genk01-UseCaseManagementSetupUseCaseMa-UseCaseManagementDLQ738693BC-fLN2YD

In [10]:
# Créons un nom plus court pour éviter les problèmes
repository_name = "genk01-bedrock-course"
print(f"✅ Nouveau nom de repository: WATTOX/{repository_name}")
print(f"✅ Cela réutilisera votre infrastructure Genk01 existante")
print(f"✅ Coût estimé: ~$0-5/mois (au lieu de $50-100/mois pour une nouvelle infra)")

✅ Nouveau nom de repository: WATTOX/genk01-bedrock-course
✅ Cela réutilisera votre infrastructure Genk01 existante
✅ Coût estimé: ~$0-5/mois (au lieu de $50-100/mois pour une nouvelle infra)


In [11]:
# Générer les commandes à exécuter dans CloudShell
repository_name = "genk01-bedrock-course"

commands = f"""
# 1. Créer la nouvelle politique de confiance
cat > trust-policy-genk01.json << 'EOF'
{{
    "Version": "2012-10-17",
    "Statement": [
        {{
            "Effect": "Allow",
            "Principal": {{
                "Federated": "arn:aws:iam::796973472728:oidc-provider/token.actions.githubusercontent.com"
            }},
            "Action": "sts:AssumeRoleWithWebIdentity",
            "Condition": {{
                "StringEquals": {{
                    "token.actions.githubusercontent.com:aud": "sts.amazonaws.com"
                }},
                "StringLike": {{
                    "token.actions.githubusercontent.com:sub": "repo:WATTOX/{repository_name}/*"
                }}
            }}
        }}
    ]
}}
EOF

# 2. Mettre à jour le rôle
aws iam update-assume-role-policy \\
    --role-name github.to.aws.oicd \\
    --policy-document file://trust-policy-genk01.json

# 3. Vérifier la mise à jour
aws iam get-role --role-name github.to.aws.oicd --query 'Role.AssumeRolePolicyDocument' --output json
"""

print("📋 COPIEZ ET EXÉCUTEZ ces commandes dans AWS CloudShell :")
print("="*60)
print(commands)
print("="*60)

📋 COPIEZ ET EXÉCUTEZ ces commandes dans AWS CloudShell :

# 1. Créer la nouvelle politique de confiance
cat > trust-policy-genk01.json << 'EOF'
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Federated": "arn:aws:iam::796973472728:oidc-provider/token.actions.githubusercontent.com"
            },
            "Action": "sts:AssumeRoleWithWebIdentity",
            "Condition": {
                "StringEquals": {
                    "token.actions.githubusercontent.com:aud": "sts.amazonaws.com"
                },
                "StringLike": {
                    "token.actions.githubusercontent.com:sub": "repo:WATTOX/genk01-bedrock-course/*"
                }
            }
        }
    ]
}
EOF

# 2. Mettre à jour le rôle
aws iam update-assume-role-policy \
    --role-name github.to.aws.oicd \
    --policy-document file://trust-policy-genk01.json

# 3. Vérifier la mise à jour
aws iam get-role --role-na

In [12]:
!cd "/content/drive/MyDrive/educational-course-content-generator-with-qna-bot-using-bedrock-main" && \
git init && \
git config user.name "WATTOX" && \
git config user.email "votre-email@gmail.com" && \
git branch -M main

hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/drive/MyDrive/educational-course-content-generator-with-qna-bot-using-bedrock-main/.git/


In [13]:
!cd "/content/drive/MyDrive/educational-course-content-generator-with-qna-bot-using-bedrock-main" && \
git remote add origin https://github.com/WATTOX/genk01-bedrock-course.git && \
git remote -v

origin	https://github.com/WATTOX/genk01-bedrock-course.git (fetch)
origin	https://github.com/WATTOX/genk01-bedrock-course.git (push)
